# 🚗 Nhận Dạng Biển Số Xe Việt Nam

## Vietnamese License Plate Recognition - Simple Version

### 🎯 Tính năng:
- Phát hiện biển số xe trong ảnh
- Nhận dạng ký tự và số trên biển số
- Hỗ trợ biển số 1 dòng và 2 dòng

### 📋 Hướng dẫn:
1. **Chọn GPU Runtime**: Runtime → Change runtime type → GPU
2. **Chạy từng cell theo thứ tự** từ trên xuống dưới
3. **Đợi cell hoàn thành** trước khi chạy cell tiếp theo

---

## 1. Clone Repository và Setup

In [ ]:
import os
import sys

# Clone repository
if not os.path.exists('BsxGoogleColab'):
    print("🔄 Cloning repository...")
    !git clone https://github.com/ngoclam99/BsxGoogleColab.git
    print("✅ Clone successful!")
else:
    print("✅ Repository already exists!")

# Change to project directory
os.chdir('BsxGoogleColab')
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
if os.path.join(os.getcwd(), 'yolov5') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'yolov5'))

print("✅ Setup complete!")

## 2. Install Dependencies từ requirements.txt

In [ ]:
# Install dependencies from requirements.txt
print("📦 Installing dependencies from requirement.txt...")
!pip install -r requirement.txt

# Install additional packages for Colab
print("📦 Installing additional Colab packages...")
!pip install -q ipywidgets

print("✅ All dependencies installed!")

## 3. Import Libraries và Check Files

In [ ]:
# Import all required libraries
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sys
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported:")
print(f"   - OpenCV: {cv2.__version__}")
print(f"   - PyTorch: {torch.__version__}")
print(f"   - NumPy: {np.__version__}")

# Import helper functions
try:
    import function.utils_rotate as utils_rotate
    import function.helper as helper
    print("✅ Helper functions imported successfully!")
except ImportError as e:
    print(f"❌ Error importing helpers: {e}")
    print("💡 Make sure repository structure is correct")

# Check required files
required_files = {
    'LP Detector Model': 'model/LP_detector.pt',
    'LP OCR Model': 'model/LP_ocr.pt', 
    'YOLOv5 Directory': 'yolov5',
    'Test Images': 'test_image',
    'Requirements': 'requirement.txt'
}

print("\n🔍 Checking required files...")
all_files_ok = True
for name, path in required_files.items():
    if os.path.exists(path):
        if os.path.isfile(path):
            size = os.path.getsize(path) / (1024*1024)  # MB
            print(f"✅ {name}: {path} ({size:.1f} MB)")
        else:
            count = len(os.listdir(path)) if os.path.isdir(path) else 0
            print(f"✅ {name}: {path} ({count} items)")
    else:
        print(f"❌ {name}: {path} - MISSING")
        all_files_ok = False

if all_files_ok:
    print("\n🟢 All required files are ready!")
else:
    print("\n🔴 Some files are missing! Check your repository.")

print("✅ Import and check complete!")

## 4. Load YOLOv5 Models

In [ ]:
# Load YOLOv5 models
print("🤖 Loading YOLOv5 models...")
print("⏳ This may take 2-3 minutes for first time...")

try:
    # Load License Plate Detection model
    print("📥 Loading License Plate Detection model...")
    yolo_LP_detect = torch.hub.load('yolov5', 'custom', 
                                   path='model/LP_detector.pt', 
                                   force_reload=True, 
                                   source='local')
    print("✅ LP Detection model loaded!")
    
    # Load License Plate OCR model
    print("📥 Loading License Plate OCR model...")
    yolo_license_plate = torch.hub.load('yolov5', 'custom', 
                                       path='model/LP_ocr.pt', 
                                       force_reload=True, 
                                       source='local')
    print("✅ LP OCR model loaded!")
    
    # Set confidence threshold
    yolo_license_plate.conf = 0.60
    print("🔧 Confidence threshold set to 0.60")
    
    print("\n🎉 ALL MODELS LOADED SUCCESSFULLY!")
    print("🟢 Ready for license plate recognition!")
    
except Exception as e:
    print(f"\n❌ ERROR LOADING MODELS: {e}")
    print("\n💡 SOLUTIONS:")
    print("   1. Runtime → Restart Runtime and run again")
    print("   2. Make sure you selected GPU runtime")
    print("   3. Check if model files exist in repository")
    print("   4. Try running cells in order from the beginning")
    
    # Set variables to None for error handling
    yolo_LP_detect = None
    yolo_license_plate = None

## 5. Detection Function

In [ ]:
def detect_license_plate(image_path):
    """Detect and recognize license plate"""
    
    # Check if models are loaded
    if 'yolo_LP_detect' not in globals() or 'yolo_license_plate' not in globals():
        print("❌ Models not loaded! Please run the model loading cell first.")
        return None, None
        
    if yolo_LP_detect is None or yolo_license_plate is None:
        print("❌ Models failed to load! Please check the model loading cell.")
        return None, None
    
    print(f"🔍 Processing: {image_path}")
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None
    
    try:
        # Detect license plates
        print("📸 Detecting license plates...")
        plates = yolo_LP_detect(img, size=640)
        list_plates = plates.pandas().xyxy[0].values.tolist()
        list_read_plates = set()
        
        print(f"🎯 Found {len(list_plates)} license plate regions")
        
        if len(list_plates) == 0:
            # Try reading entire image
            print("🔄 Trying to read entire image...")
            lp = helper.read_plate(yolo_license_plate, img)
            if lp != "unknown":
                list_read_plates.add(lp)
        else:
            # Process each detected plate
            for i, plate in enumerate(list_plates):
                print(f"📝 Reading plate {i+1}/{len(list_plates)}...")
                
                x = int(plate[0])
                y = int(plate[1])
                w = int(plate[2] - plate[0])
                h = int(plate[3] - plate[1])
                
                # Crop license plate
                crop_img = img[y:y+h, x:x+w]
                
                # Draw rectangle
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                # Try different image processing
                found = False
                for cc in range(0, 2):
                    for ct in range(0, 2):
                        try:
                            processed_img = utils_rotate.deskew(crop_img, cc, ct)
                            lp = helper.read_plate(yolo_license_plate, processed_img)
                            if lp != "unknown":
                                list_read_plates.add(lp)
                                found = True
                                break
                        except Exception as e:
                            continue
                    if found:
                        break
        
        # Add text to image
        if list_read_plates:
            y_offset = 30
            for plate_text in list_read_plates:
                cv2.putText(img, str(plate_text), (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                y_offset += 40
        
        if list_read_plates:
            print(f"✅ Result: {list_read_plates}")
        else:
            print("❌ No license plate recognized")
        
        return img, list_read_plates
        
    except Exception as e:
        print(f"❌ Error during detection: {e}")
        return None, None

print("✅ Detection function ready!")

## 6. Test với Ảnh Mẫu

In [ ]:
# Find test images
test_images = glob.glob("test_image/*.jpg") + glob.glob("test_image/*.png") + glob.glob("test_image/*.jpeg")

if test_images:
    print(f"🖼️ Found {len(test_images)} test images:")
    for i, img in enumerate(test_images):
        print(f"   {i}: {img}")
    
    # Test first image (change index to test different images)
    test_index = 0
    
    if test_index < len(test_images):
        test_img = test_images[test_index]
        
        print(f"\n🚀 Testing: {test_img}")
        print("=" * 50)
        
        result_img, detected_plates = detect_license_plate(test_img)
        
        if result_img is not None:
            print("\n" + "=" * 50)
            print(f"🎯 FINAL RESULT: {detected_plates if detected_plates else 'NO DETECTION'}")
            print("=" * 50)
            
            # Display results
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Original image
            original = cv2.imread(test_img)
            axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            axes[0].set_title("🖼️ Original Image", fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            # Result image
            axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            result_title = f"🎯 Result: {detected_plates if detected_plates else 'No detection'}"
            axes[1].set_title(result_title, fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Summary
            print(f"\n📊 SUMMARY:")
            print(f"   📁 File: {test_img}")
            print(f"   📏 Size: {original.shape[1]}x{original.shape[0]}")
            print(f"   🔢 Plates found: {len(detected_plates) if detected_plates else 0}")
            if detected_plates:
                for i, plate in enumerate(detected_plates, 1):
                    print(f"   🚗 Plate {i}: {plate}")
        else:
            print("❌ Failed to process image")
    else:
        print(f"❌ Invalid index {test_index}. Choose from 0 to {len(test_images)-1}")
else:
    print("❌ No test images found in test_image/ directory")
    print("💡 Make sure your repository has test images")

## 7. Upload Ảnh Của Bạn

In [ ]:
from google.colab import files

def upload_and_test():
    """Upload and test your own image"""
    print("📤 Select image file from your computer...")
    print("💡 Supported formats: .jpg, .jpeg, .png")
    
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Processing uploaded image: {filename}")
        print("=" * 50)
        
        # Save uploaded file
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])
        
        # Process image
        result_img, detected_plates = detect_license_plate(filename)
        
        if result_img is not None:
            print("\n" + "=" * 50)
            print(f"🎯 RESULT: {detected_plates if detected_plates else 'NO DETECTION'}")
            print("=" * 50)
            
            # Display results
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Original
            original = cv2.imread(filename)
            axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            axes[0].set_title("🖼️ Your Image", fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            # Result
            axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            result_title = f"🎯 Result: {detected_plates if detected_plates else 'No detection'}"
            axes[1].set_title(result_title, fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Summary
            print(f"\n📊 SUMMARY:")
            print(f"   📁 File: {filename}")
            print(f"   📏 Size: {original.shape[1]}x{original.shape[0]}")
            if detected_plates:
                for i, plate in enumerate(detected_plates, 1):
                    print(f"   🚗 License Plate {i}: {plate}")
            else:
                print("   ❌ No license plate detected")
                print("   💡 Try with a clearer image or different angle")
        else:
            print("❌ Failed to process uploaded image")

# Uncomment the line below to upload and test your image
# upload_and_test()

## 📝 Hướng Dẫn Sử Dụng

### 🚀 Cách sử dụng:
1. **Chạy Cell 1**: Clone repository và setup
2. **Chạy Cell 2**: Install dependencies từ requirement.txt
3. **Chạy Cell 3**: Import libraries và check files
4. **Chạy Cell 4**: Load YOLOv5 models (đợi 2-3 phút)
5. **Chạy Cell 5**: Define detection function
6. **Chạy Cell 6**: Test với ảnh mẫu
7. **Chạy Cell 7**: Upload ảnh của bạn (uncomment dòng cuối)

### 💡 Tips để có kết quả tốt:
- **Chọn GPU Runtime** để xử lý nhanh hơn
- **Sử dụng ảnh rõ nét** và chất lượng cao
- **Biển số không bị che khuất** hoặc mờ
- **Góc chụp thẳng** cho kết quả tốt nhất
- **Ánh sáng đủ** để nhìn rõ ký tự

### 🔧 Khắc phục sự cố:
- **Models không load được**: 
  - Runtime → Restart Runtime và chạy lại
  - Đảm bảo đã chọn GPU runtime
- **Lỗi import**: Kiểm tra cấu trúc repository
- **Không nhận dạng được**: 
  - Thử ảnh khác rõ nét hơn
  - Kiểm tra biển số có bị che không
  - Thử góc chụp khác

### ✨ Ưu điểm của version này:
- **Sử dụng requirement.txt** - Chuyên nghiệp và đơn giản
- **Cài đặt tự động** - Không cần install thủ công từng package
- **Ít lỗi hơn** - Dependencies được quản lý tốt
- **Dễ maintain** - Dễ cập nhật và bảo trì

---
**🚗 Chúc bạn nhận dạng biển số thành công! 🎉**